In [ ]:
!git clone https://github.com/indra622/tiny_sgspeech

fatal: destination path 'tiny_sgspeech' already exists and is not an empty directory.


In [ ]:
!wget https://www.openslr.org/resources/12/train-clean-100.tar.gz
!wget https://www.openslr.org/resources/12/test-clean.tar.gz
!wget https://www.openslr.org/resources/12/dev-clean.tar.gz

--2021-06-28 06:32:08--  https://www.openslr.org/resources/12/train-clean-100.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6387309499 (5.9G) [application/x-gzip]
Saving to: ‘train-clean-100.tar.gz.1’

train-clean-100.tar  25%[====>               ]   1.49G  19.8MB/s    eta 3m 56s 

In [ ]:
!tar -xvzf train-clean-100.tar.gz && rm train-clean-100.tar.gz
!tar -xvzf test-clean.tar.gz && rm test-clean.tar.gz
!tar -xvzf dev-clean.tar.gz && rm dev-clean.tar.gz

In [ ]:
!python tiny_sgspeech/create_librispeech_trans.py --dir /content/LibriSpeech/train-clean-100 /content/LibriSpeech/train-clean-100/transcripts.tsv
!python tiny_sgspeech/create_librispeech_trans.py --dir /content/LibriSpeech/dev-clean /content/LibriSpeech/dev-clean/transcripts.tsv
!python tiny_sgspeech/create_librispeech_trans.py --dir /content/LibriSpeech/test-clean /content/LibriSpeech/test-clean/transcripts.tsv

In [ ]:
!cd tiny_sgspeech && pip install .

In [ ]:
import os
import math
import argparse
from tiny_sgspeech.sgspeech.utils import setup_environment, setup_strategy

setup_environment()
import tensorflow as tf

tf.keras.backend.clear_session()

strategy = setup_strategy([0])

from tiny_sgspeech.sgspeech.configs.config import Config
from tiny_sgspeech.sgspeech.datasets.speech_dataset import SpeechSliceDataset
from tiny_sgspeech.sgspeech.featurizers.speech_featurizer import NumpySpeechFeaturizer
from tiny_sgspeech.sgspeech.featurizers.text_featurizer import CharFeaturizer
from tiny_sgspeech.sgspeech.runners.transducer_runners import TransducerTrainer
from tiny_sgspeech.sgspeech.models.conformer import Conformer
from tiny_sgspeech.sgspeech.optimizers.schedules import TransformerSchedule

config = Config('/content/tiny_sgspeech/config.yml')
speech_featurizer = NumpySpeechFeaturizer(config.speech_config)
text_featurizer = CharFeaturizer(config.decoder_config)

train_dataset = SpeechSliceDataset(
    speech_featurizer=speech_featurizer, text_featurizer=text_featurizer,
    **vars(config.learning_config.train_dataset_config)
)
eval_dataset = SpeechSliceDataset(
    speech_featurizer=speech_featurizer, text_featurizer=text_featurizer,
    **vars(config.learning_config.eval_dataset_config)
)

conformer_trainer = TransducerTrainer(
    config=config.learning_config.running_config,
    text_featurizer=text_featurizer, strategy=strategy
)

with conformer_trainer.strategy.scope():
    # build model
    conformer = Conformer(**config.model_config, vocabulary_size=text_featurizer.num_classes)
    conformer._build(speech_featurizer.shape)
    conformer.summary(line_length=120)

    optimizer_config = config.learning_config.optimizer_config
    optimizer = tf.keras.optimizers.Adam(
        TransformerSchedule(
            d_model=conformer.dmodel,
            warmup_steps=optimizer_config["warmup_steps"],
            max_lr=(0.05 / math.sqrt(conformer.dmodel))
        ),
        beta_1=optimizer_config["beta1"],
        beta_2=optimizer_config["beta2"],
        epsilon=optimizer_config["epsilon"]
    )

conformer_trainer.compile(model=conformer, optimizer=optimizer,
                          max_to_keep=10)

conformer_trainer.fit(train_dataset, eval_dataset, train_bs=4, eval_bs=1)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/conformer/tensorboard'